In [1]:
import json
import os
import pickle
import random
import sys
import pickle

from typing import Callable, Dict, List, Optional
import haiku as hk
import ase
import ase.io
import jax
import jax.numpy as jnp
import numpy as np
import optax
import yaml

from phonax.datasets import (
    datasets,
    ph_datasets,
)
from phonax.optimizer import optimizer
from phonax.energy_force_train import energy_force_train
from phonax.loss import (
    WeightedEnergyFrocesStressLoss,
    crystalHessianLoss,
)
from phonax.nequip_model import NequIP_JAXMD_model

from phonax.utils import (
    create_directory_with_random_name,
)

from phonax.phonons_train import (
    ph_evaluate,
    hessian_train,
    two_stage_hessian_train,
)

from phonax.predictors import (
    predict_energy_forces_stress,
    predict_crystal_hessian,
)
from phonax.phonons import (
    atoms_to_ext_graph, 
    predict_gamma_spectra_filter
)

from phonax.data_utils import to_f32

from phonax.trained_models import NequIP_JAXMD_molecule_model   

jax.config.update("jax_debug_nans", True)
jax.config.update("jax_debug_infs", True)
np.set_printoptions(precision=3, suppress=True)

# Introduction

Previously, we have been making the second derivative Hessian matrix predictions given an energy model, which is trained with zeroth and first order derivative data (i.e. energy and forces).
Here in this tutorial, we will demonstrate an alternative view of the Hessian data, and use them as part of the training data (the Hessian matrix, or the eigenvalue spectrum) for the periodic crystal solids.

As discussed in our work, this augments the training dataset in converging the energy models. Locally, the second derivative data improves the local curvatures of the energy model landscape.
To demonstrate this here, we use periodic crystal cases as the examples, and focus on using the Hessian matrix as an extended data type to train the energy model/functional. 


## Energy model training with crystalline Hessian data

This section focuses on using the Hessian matrix as an extended data type to train the energy model/functional for the interatomic potentials.
The training procedure can be viewed as an extension to the conventional interatomic potential training using only the zeroth and first order training data for the energy landscape (i.e. energy and force).

For the specific example, we use Si crystal (mp-149) example as discussed in our work.
When trained with energy and force data for the energy model, we found the training can be improved by the augmented supercell geometries, which effectively create finer mesh grid in the momentum space sampling.
However, in our training example here, we will only use the $1 \times 1 \times 1$ energy and force data, combined with the crystal Hessian data to achieve good phonon predictions, without supercell geometries for the energy and forces.

Compared to the molecular cases, the derivation is more complicated for the periodic crystalline solids.
This is due to the periodic boundary conditions which require the construction for the extended graphs in the computations.
To describe the phonon states, one has to introduce crystal momentum $\vec{k}$ as well from the Bloch theorem.
In summary, the Hessian training involves the random selection for momentum $\vec{k}$, and two (random) displacement patterns at this momentum. 
The one can project out the dynamical matrix element given these two displacement patterns. (In analogy to the molecular cases, one can also choose random atomic sites and two displacement vectors on these sites to perform the projection.)
With enough projections (or slices) of the Hessians from the trained model, the energy model would converge to the groundtruth energy functional at all momentum $\vec{k}$ and bewteen any states.

One note about the random direction (or patterns) here. If the displacements are constrained to the x, y or z direcitons, the specific coordinate frame breaks the (rotational) equivariance in the training procedure, despite our energy model respects the equivarnace. The randomized projections restore the rotational equivariance.





### Initialize the training procedures

Here we load the training configurations and datasets, initialize the model, loss functions, and the predictors used for the next training steps.

In [2]:
# Load the config files
with open('data/mp-149/mp149-fh.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
        
if 'save_dir_name' in config:
    save_dir_name = config['save_dir_name']
else:
    save_dir_name = create_directory_with_random_name()
print(save_dir_name)
        
# Save config
with open(f"{save_dir_name}/config.yaml", "w") as f:
    yaml.dump(config, f)
    
# Load the energy/force training/valid dataset
EF_train_loader, EF_valid_loader, EF_test_loader, EF_r_max = datasets(
    r_max = config["cutoff"],
    config_dataset = config["dataset"],
)

# Load the Hessian training/valid dataset
ph_train_loader, ph_valid_loader, ph_test_loader, ph_r_max, num_message_passing = ph_datasets(
    r_max = config["cutoff"],
    config_dataset = config["ph_dataset"],
    num_message_passing = config['model']['num_layers'],
)

assert ph_r_max==EF_r_max
r_max = EF_r_max

# Initialize the NequIP model
model_fn, params, num_message_passing = NequIP_JAXMD_model(
    r_max=r_max,
    atomic_energies_dict={},
    train_graphs=EF_train_loader.graphs,
    initialize_seed=config["model"]["seed"],
    num_species = config["model"]["num_species"],
    use_sc = True,
    graph_net_steps = config["model"]["num_layers"],
    hidden_irreps = config["model"]["internal_irreps"],
    nonlinearities =  {'e': 'swish', 'o': 'tanh'},
    save_dir_name = save_dir_name,
    reload = config["initialization"]['reload'] if 'reload' in config["initialization"] else None,
)
    
print("num_params:", sum(p.size for p in jax.tree_util.tree_leaves(params)))

# Define Loss functions for energy/force and Hessians
EF_loss_fn = WeightedEnergyFrocesStressLoss(
    energy_weight = config["training"]["energy_weight"],
    forces_weight = config["training"]["forces_weight"],
    stress_weight = config["training"]["stress_weight"],
)


H_loss_fn = crystalHessianLoss()

# Predictors for the energy/force/stress and Hessians
EF_predictor = jax.jit(
    lambda w, g: predict_energy_forces_stress(lambda *x: model_fn(w, *x), g)
)

H_predictor = jax.jit(
    lambda w, g: predict_crystal_hessian(lambda *x: model_fn(w, *x), g)
)


2024-03-17-20:48-nice-katrine
2024-03-17-20:48-nice-katrine
nums check 1500 9000 128
Loaded 1000 training configurations from 'data/mp-149/mp-149-SC111_train.xyz'
Loaded 200 validation configurations from 'data/mp-149/mp-149-SC333_valid.xyz'
Total number of configurations: train=1000, valid=200, test=0


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 736.38it/s]
0it [00:00, ?it/s]
Pad valid: 100%|█████████████████████████████| 100/100 [00:00<00:00, 305.25it/s]


Total number of configurations: train=2000, valid=100, test=0
Compute the average number of neighbors: 27.992
Do not normalize the radial basis (avg_r_min=None)
Computed average Atomic Energies using least squares: {14: -5.531418045215}
Create NequIP (JAX-MD version) with parameters {'use_sc': True, 'graph_net_steps': 2, 'hidden_irreps': '16x0e + 16x0o +12x1e + 12x1o + 8x2e +8x2o', 'nonlinearities': {'e': 'swish', 'o': 'tanh'}, 'r_max': 5.0, 'avg_num_neighbors': 27.992, 'avg_r_min': None, 'num_species': 100, 'radial_basis': <function bessel_basis at 0x7ff21a0435e0>, 'radial_envelope': <function soft_envelope at 0x7ff21a043670>}


2024-03-17 20:49:00.392487: W external/xla/xla/service/gpu/nvptx_compiler.cc:742] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.4.99). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


num_params: 58920


### Training with energy / force training data

In this section, we demonstrate the energy model training by using only the energy and force training data, as is done in the conventional interatomic potential model training procedure.
Note in this simple propane molecule example, we only use four molecular configurations in the training dataset.



In [3]:
# force only training

Fonly_params = params.copy()

gradient_transform, steps_per_interval, max_num_intervals = optimizer(
    lr = config["training"]["learning_rate"],
    max_num_intervals = config["training"]["max_num_intervals"],
    steps_per_interval = config["training"]["steps_per_interval"],
    # weight_decay = config["training"]["weight_decay"],
)
optimizer_state = gradient_transform.init(params)
print("optimizer num_params:", sum(p.size for p in jax.tree_util.tree_leaves(optimizer_state)))
    
train_out = energy_force_train(
    EF_predictor,
    Fonly_params,
    optimizer_state,
    EF_train_loader,
    EF_valid_loader,
    EF_test_loader,
    gradient_transform,
    EF_loss_fn,
    max_num_intervals,
    steps_per_interval,
    save_dir_name,
    ema_decay = config["training"]["ema_decay"],
    patience = config["training"]["patience"],
)



optimizer num_params: 176762
Started training


eval_train:   0%|                                         | 0/7 [00:00<?, ?it/s]

Compiled function `model` for args:
cache size: 1


eval_train: 100%|██████████████████████████| 7/7 [00:09<00:00,  1.29s/it, n=889]


Interval 0: eval_train: loss=354.1589, mae_e_per_atom=2768.2 meV, mae_f=1523.9 meV/Å, mae_s=27.9 meV/Å³


...

Interval 14: eval_valid: loss=2.0768, mae_e_per_atom=18.4 meV, mae_f=133.1 meV/Å, mae_s=19.8 meV/Å³


eval_train: 100%|██████████████████████████| 7/7 [00:00<00:00, 57.83it/s, n=889]


Interval 15: eval_train: loss=0.0240, mae_e_per_atom=2.5 meV, mae_f=14.1 meV/Å, mae_s=57.0 meV/Å³


eval_valid: 100%|████████████████████████| 40/40 [00:00<00:00, 65.10it/s, n=200]


Interval 15: eval_valid: loss=2.1226, mae_e_per_atom=18.5 meV, mae_f=134.6 meV/Å, mae_s=20.0 meV/Å³
Training complete


### Training with crystal Hessians

In the next section, we show the energy model training with additional second order Hessian data beynd the force data at the first order derivative.
Specifically, there are two tages in the training procedure.
The first stage, or the warm-up stage, uses only the energy and force data to initialize the energy model training.
After trained with several warm-up steps, the second training stage adds the Hessian data in computing the loss function.
At the second stage, the update gradients come from both the energy / force training losses, and the Hessian losses, and we combine the two (weighted sum) to get the overall training gradient.

From the training results below, one can see the valid force MAE further decreases once we have included the Hessian training data in the second stage.

We note that, it remains an open question to find the optimal training procedure in adding the second order Hessian data. For example, one can perform the training without warm-up steps, or gradually ramp up the training weights associated with the Hessian losses.
It would be relevant to examine the energy model converged for their predictions and stability under molecular dynamics simulations.


In [9]:
FH_params = params.copy()

# restart the optimizer state
gradient_transform, steps_per_interval, max_num_intervals = optimizer(
    lr = config["training"]["learning_rate"],
    max_num_intervals = 20, #config["training"]["max_num_intervals"],
    steps_per_interval = 100, # config["training"]["steps_per_interval"],
    # weight_decay = config["training"]["weight_decay"],
)
optimizer_state = gradient_transform.init(params)
print("optimizer num_params:", sum(p.size for p in jax.tree_util.tree_leaves(optimizer_state)))

two_stage_hessian_train(
    energy_forces_stress_predictor=EF_predictor,
    phonon_predictor=H_predictor,
    params=FH_params,
    gradient_transform=gradient_transform,
    optimizer_state=optimizer_state,
    steps_per_interval=steps_per_interval,
    EF_loss_fn=EF_loss_fn,
    H_loss_fn=H_loss_fn,
    EF_train_loader=EF_train_loader,
    EF_valid_loader=EF_valid_loader,
    H_train_loader=ph_train_loader,
    H_valid_loader=ph_valid_loader,
    warmup_num_intervals=10 ,#config['training']['warmup_num_intervals'],
    phtrain_num_intervals=25  ,#config['training']['phtrain_num_intervals'],
    periodic_crystal=True,
)









optimizer num_params: 176762


eval_train: 100%|██████████████████████████| 7/7 [00:00<00:00, 54.72it/s, n=889]


Interval 0: eval_train: loss=350.4000, mae_e_per_atom=2766.672 meV, mae_f=1520.624 meV/Å, mae_s=27.592 meV/Å³


eval_valid: 100%|████████████████████████| 40/40 [00:00<00:00, 61.54it/s, n=200]


Interval 0: eval_valid: loss=310.6482, mae_e_per_atom=2762.956 meV, mae_f=1373.478 meV/Å, mae_s=9.123 meV/Å³


...

Evaluating hessian train: 100%|█| 1000/1000 [00:07<00:00, 127.20it/s, n_graphs=2


interval 24: hessian train hessian MAE 0.227 eV/A2 


Evaluating hessian valid: 100%|██| 50/50 [00:00<00:00, 135.60it/s, n_graphs=100]


interval 24: hessian valid hessian MAE 0.210 eV/A2 


Interval 24: 100%|███████████████| 100/100 [00:02<00:00, 37.38it/s, loss=0.0193]
